In [2]:
import glob
import pandas as pd
import os
import re
from bs4 import BeautifulSoup
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
os.chdir("..")

In [4]:
import glob
import re
from bs4 import BeautifulSoup

sgm_files = glob.glob("reuters21578/*.sgm")  # Ensure `dir` is defined

articles_data = []

for file_path in sgm_files:
    print(file_path)  # Print file name for debugging

    with open(file_path, "r", encoding="ISO-8859-1") as f:
        soup = BeautifulSoup(f.read(), "html.parser")

        for reuters in soup.find_all("reuters"):
            body = reuters.body.get_text(strip=True) if reuters.body else ""
            text = body.strip()

            topics_tag = reuters.find("topics")
            topics = [topic.get_text(strip=True) for topic in topics_tag.find_all("d")] if topics_tag else []
            topics = ",".join(topics)

            split = reuters.get("lewissplit", "").upper()

            text = re.sub(r'&#\d+;', '', text)  # Optional: Use `html.unescape(text)`

            if text:  
                articles_data.append({"topics": topics, "text": text, "split": split})


reuters21578/reut2-000.sgm
reuters21578/reut2-001.sgm
reuters21578/reut2-002.sgm
reuters21578/reut2-003.sgm
reuters21578/reut2-004.sgm
reuters21578/reut2-005.sgm
reuters21578/reut2-006.sgm
reuters21578/reut2-007.sgm
reuters21578/reut2-008.sgm
reuters21578/reut2-009.sgm
reuters21578/reut2-010.sgm
reuters21578/reut2-011.sgm
reuters21578/reut2-012.sgm
reuters21578/reut2-013.sgm
reuters21578/reut2-014.sgm
reuters21578/reut2-015.sgm
reuters21578/reut2-016.sgm
reuters21578/reut2-017.sgm
reuters21578/reut2-018.sgm
reuters21578/reut2-019.sgm
reuters21578/reut2-020.sgm
reuters21578/reut2-021.sgm


In [5]:
df = pd.DataFrame(articles_data)
df

,topics,text,split
0,cocoa,Showers continued throughout the week in\nthe ...,TRAIN
1,,Standard Oil Co and BP North America\nInc said...,TRAIN
2,,Texas Commerce Bancshares Inc's Texas\nCommerc...,TRAIN
3,,BankAmerica Corp is not under\npressure to act...,TRAIN
4,"grain,wheat,corn,barley,oat,sorghum",The U.S. Agriculture Department\nreported the ...,TRAIN
...,...,...,...
19038,ship,The Japan/India-Pakistan-Gulf/Japan\nshipping ...,TEST
19039,ipi,The Soviet Union's industrial output is\ngrowi...,TEST
19040,gold,Six black miners have been killed\nand two inj...,TEST
19041,,The prospect of a dominant alliance of\nsocial...,TEST


In [6]:
df.dropna()
df["split"].unique()

array(['TRAIN', 'NOT-USED', 'TEST'], dtype=object)

In [7]:
train_df = df[df['split'] == "TRAIN"]
train_df

,topics,text,split
0,cocoa,Showers continued throughout the week in\nthe ...,TRAIN
1,,Standard Oil Co and BP North America\nInc said...,TRAIN
2,,Texas Commerce Bancshares Inc's Texas\nCommerc...,TRAIN
3,,BankAmerica Corp is not under\npressure to act...,TRAIN
4,"grain,wheat,corn,barley,oat,sorghum",The U.S. Agriculture Department\nreported the ...,TRAIN
...,...,...,...
13016,ship,Australian trade unions said they have\nlaunch...,TRAIN
13017,,The ruling Liberal Democratic Party (LDP)\nhas...,TRAIN
13018,,Ministry of International Trade and\nIndustry ...,TRAIN
13019,,Sharp Corp will set up a 100 pct-owned\nsubsid...,TRAIN


In [8]:
test_df = df[df['split'] == "TEST"]
test_df

,topics,text,split
13020,,Yugoslav government plans to stop\nsubsidising...,TEST
13021,,"The French government won, as expected, a\nvot...",TEST
13022,trade,Mounting trade friction between the\nU.S. And ...,TEST
13023,,"The union representing 7,000 pilots at\nUnited...",TEST
13024,grain,A survey of 19 provinces and seven cities\nsho...,TEST
...,...,...,...
19038,ship,The Japan/India-Pakistan-Gulf/Japan\nshipping ...,TEST
19039,ipi,The Soviet Union's industrial output is\ngrowi...,TEST
19040,gold,Six black miners have been killed\nand two inj...,TEST
19041,,The prospect of a dominant alliance of\nsocial...,TEST


In [9]:
#instantiate CountVectorizer() 
cv=CountVectorizer() 

# this steps generates word counts for the words in your docs 
word_count_vector=cv.fit_transform(df["text"])

In [10]:
word_count_vector

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 1510442 stored elements and shape (19043, 41633)>

In [11]:
df.shape

(19043, 3)

In [12]:
import pandas as pd 
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import CountVectorizer 

In [13]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [14]:
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names_out(), columns=["text"])
df_idf

,text
00,4.938305
000,2.399402
0000,7.858775
00000,10.161360
0002,10.161360
...,...
zuyuan,10.161360
zverev,10.161360
zweig,10.161360
zwermann,10.161360
